Imports added for web scraping

In [1]:
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page
import pandas as pd

Web page containing Scorecard links read

In [2]:
match_schedule_html = open('match_schedule_html.txt', 'r')
soup = BeautifulSoup(match_schedule_html, 'lxml')
match_schedule_html.close()

Scorecard links extracted

In [21]:
links = soup.select('a')
scorecards = [scorecard['href'] for scorecard in links]
scorecards_links = []
for link in scorecards:
  if ('Scorecard.aspx' in link):
    scorecards_links.append(link)
scorecards_links.pop(0)
scorecards_links

['Scorecard.aspx?MatchID=1830',
 'Scorecard.aspx?MatchID=1828',
 'Scorecard.aspx?MatchID=1770',
 'Scorecard.aspx?MatchID=1829',
 'Scorecard.aspx?MatchID=1769',
 'Scorecard.aspx?MatchID=1771',
 'Scorecard.aspx?MatchID=1826',
 'Scorecard.aspx?MatchID=1827',
 'Scorecard.aspx?MatchID=1772',
 'Scorecard.aspx?MatchID=1834',
 'Scorecard.aspx?MatchID=1831',
 'Scorecard.aspx?MatchID=1835',
 'Scorecard.aspx?MatchID=1773',
 'Scorecard.aspx?MatchID=1776',
 'Scorecard.aspx?MatchID=1775',
 'Scorecard.aspx?MatchID=1832',
 'Scorecard.aspx?MatchID=1774',
 'Scorecard.aspx?MatchID=1833',
 'Scorecard.aspx?MatchID=1839',
 'Scorecard.aspx?MatchID=1777',
 'Scorecard.aspx?MatchID=1780',
 'Scorecard.aspx?MatchID=1840',
 'Scorecard.aspx?MatchID=1778',
 'Scorecard.aspx?MatchID=1838',
 'Scorecard.aspx?MatchID=1779',
 'Scorecard.aspx?MatchID=1836',
 'Scorecard.aspx?MatchID=1842',
 'Scorecard.aspx?MatchID=1841',
 'Scorecard.aspx?MatchID=1845',
 'Scorecard.aspx?MatchID=1783',
 'Scorecard.aspx?MatchID=1843',
 'Scorec